In [25]:
from sklearn import datasets
from sklearn.model_selection import train_test_split, KFold
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, classification_report
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, BatchNormalization
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from keras.callbacks import EarlyStopping
from keras.regularizers import l1_l2
import numpy as np


Função para criar modelos

In [35]:
# Função para criar o modelo
def criar_modelo(input_shape=4, regularization_rate=0.01):
    modelo = Sequential()
    modelo.add(Input(shape=(input_shape,)))
    
    # Primeira camada oculta
    modelo.add(Dense(units=5, activation='relu', 
                    kernel_regularizer=l1_l2(regularization_rate)))
    modelo.add(BatchNormalization())
    modelo.add(Dropout(0.2))
    
    # Segunda camada oculta
    modelo.add(Dense(units=4, activation='relu',
                    kernel_regularizer=l1_l2(regularization_rate)))
    modelo.add(BatchNormalization())
    modelo.add(Dropout(0.2))
    
    # Camada de saída
    modelo.add(Dense(units=3, activation="softmax"))
    
    modelo.compile(optimizer="adam", 
                  loss="categorical_crossentropy", 
                  metrics=["accuracy"])
    
    return modelo

Recebendo e tratando dados

In [36]:
base = datasets.load_iris()
previsores = base.data
classe = base.target

#Passando os dados da classe para one hot encoding
classe_dummy = to_categorical(classe)

Criando e configurando o treinamento

In [37]:
# Configuração do K-Fold
kfold = KFold(n_splits=10, shuffle=True, random_state=0)
historicos = []
accuracies = []
f1_scores = []

#Loop principal do Fold
for fold, (treino_idx, teste_idx) in enumerate(kfold.split(previsores), 1):
# Separar dados de treino e teste
    x_treinamento, x_teste = previsores[treino_idx], previsores[teste_idx]
    y_treinamento, y_teste = classe_dummy[treino_idx], classe_dummy[teste_idx]

Normalizar os dados do treinamento

In [38]:
# Normalização dos dados
scaler = StandardScaler()
x_treinamento = scaler.fit_transform(x_treinamento)
x_teste = scaler.transform(x_teste)

Early Stopping configuration

In [39]:
# Early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=50,
    restore_best_weights=True,
    verbose=1
)

Criação da estrutura da rede neural com classe Sequencial

In [ ]:
 # Criar e treinar o modelo
modelo = criar_modelo()
historico = modelo.fit(
    x_treinamento, y_treinamento,
        epochs=100,
        validation_data=(x_teste, y_teste),
        verbose=2,
        callbacks=[early_stopping]
    )

Fazer previsões

In [ ]:
# Fazer previsões
previsoes = modelo.predict(x_teste)
y_teste_matrix = [np.argmax(t) for t in y_teste]
y_previsoes_matrix = [np.argmax(t) for t in previsoes]

Calcular métricas

In [ ]:
    # Calcular métricas
accuracy = accuracy_score(y_teste_matrix, y_previsoes_matrix)
f1 = f1_score(y_teste_matrix, y_previsoes_matrix, average='weighted')
    
print(f'Accuracy: {accuracy:.4f}')
print(f'F1-Score: {f1:.4f}')
    
    # Armazenar resultados
historicos.append(historico.history)
accuracies.append(accuracy)
f1_scores.append(f1)

# Plotar resultados
plt.figure(figsize=(15, 5))

# Gráfico de accuracy durante treinamento do último fold
plt.subplot(1, 2, 1)
plt.plot(historicos[-1]['accuracy'], label='Treino')
plt.plot(historicos[-1]['val_accuracy'], label='Validação')
plt.title('Acurácia do Modelo (Último Fold)')
plt.ylabel('Acurácia')
plt.xlabel('Época')
plt.legend()

# Box plot das acurácias de todos os folds
plt.subplot(1, 2, 2)
plt.boxplot(accuracies)
plt.title('Distribuição das Acurácias')
plt.ylabel('Acurácia')
plt.xlabel('Folds')

plt.tight_layout()
plt.show()

# Imprimir resultados finais
print("\nResultados Finais:")
print(f"Média da Acurácia: {np.mean(accuracies):.4f} (±{np.std(accuracies):.4f})")
print(f"Média do F1-Score: {np.mean(f1_scores):.4f} (±{np.std(f1_scores):.4f})")


Matriz de confusão

In [ ]:
# Criar matriz de confusão com o último fold
confusao = confusion_matrix(y_teste_matrix, y_previsoes_matrix)
plt.figure(figsize=(8, 6))
sns.heatmap(confusao, annot=True, fmt="d", cmap="Blues",
            xticklabels=base.target_names,
            yticklabels=base.target_names)
plt.xlabel("Classe Predita")
plt.ylabel("Classe Real")
plt.title("Matriz de Confusão (Último Fold)")
plt.show()